In [1]:
import gurobipy as gp
from gurobipy import GRB
from eventlet.timeout import Timeout

# import auxillary packages
import requests  # for loading the example source code
import openai
import json

# import flaml and autogen
from flaml import autogen
from flaml.autogen.agentchat import Agent, UserProxyAgent
from Agent import ChargingAgentAll, ChargingAgentIndividual, reset_params_file, clear_param_backups
from flaml.autogen.code_utils import extract_code

In [2]:
# global params_filepath
params_filepath = "Model/params/EVCharging.json"
params_filepath_backup = "Model/params/EVCharging_original.json"

In [3]:
reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

In [4]:
config_list = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-4"],
    },
)

In [5]:
config_list_gpt3_5 = autogen.config_list_from_json(
    env_or_file = "OAI_CONFIG_LIST",
    filter_dict={
        "model": ["gpt-3.5-turbo-16k"],
    },
)

In [6]:
autogen.oai.ChatCompletion.start_logging()

In [7]:
code_path = "Model/EVCharging.py"

In [8]:
with open(code_path) as f:
    code = f.read()

In [9]:
example_qa = """
----------
Instruction: Process this request for parking lot manager
Change carbon cost to 30
Answer Code:
```JSON
"lot_params":{"carbon_cost": 30}
```

----------
Instruction: Process this request for car ZNV0672
Update end time to 10 PM
Answer Code:
```JSON
"ZNV0672":{"end_time": 22}
```
"""

In [10]:
agentAll = ChargingAgentAll(
    name="Tesla Charging Example",
    source_code=code,
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0x2ca4d787
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -135.7200000
Presolve removed 56 rows and 192 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: -142.04 -135.72 
No other solutions better than -142.04

Optimal solution found (tolerance 1.00e-04)
Best objective -1.420400000000e+02, best bound -1.420400000000e+02, gap 0.0

In [11]:
agentIndividual = ChargingAgentIndividual(
    name="Tesla Charging Example",
    source_code=code,
    example_qa=example_qa,
    json_filepath=params_filepath,
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0x2ca4d787
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -135.7200000
Presolve removed 56 rows and 192 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: -142.04 -135.72 
No other solutions better than -142.04

Optimal solution found (tolerance 1.00e-04)
Best objective -1.420400000000e+02, best bound -1.420400000000e+02, gap 0.0000%
got status


In [12]:
user_manager = UserProxyAgent(
    "manager", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

user_car1 = UserProxyAgent(
    "HVC0538", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

user_car2 = UserProxyAgent(
    "ZNV0672", max_consecutive_auto_reply=0,
    human_input_mode="NEVER", code_execution_config=False
)

In [13]:
user_manager.initiate_chat(agentAll, message="Change energy cost weight to 1", clear_history=True)

manager (to Tesla Charging Example):

Change energy cost weight to 1

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"lot_params":{"elec_cost_weight": 1}
```

--------------------------------------------------------------------------------
{'lot_params': {'elec_cost_weight': 1}}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0x488cb761
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -124.200

In [14]:
reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

In [15]:
user_manager.initiate_chat(agentIndividual, message="Change energy cost weight to 1", clear_history=True)

manager (to Tesla Charging Example):

Change energy cost weight to 1

--------------------------------------------------------------------------------
{'elec_cost': {'0': 0.3, '1': 0.3, '2': 0.3, '3': 0.3, '4': 0.3, '5': 0.3, '6': 0.3, '7': 0.35, '8': 0.35, '9': 0.35, '10': 0.35, '11': 0.3, '12': 0.3, '13': 0.3, '14': 0.3, '15': 0.3, '16': 0.3, '17': 0.3, '18': 0.3, '19': 0.35, '20': 0.35, '21': 0.35, '22': 0.3, '23': 0.3}, 'elec_cost_desc': 'USD per KWH each hour of day', 'carbon_cost': 25, 'carbon_cost_desc': 'carbon emissions(grams) per KWH each hour of day', 'carbon_cost_weight': 0.3, 'carbon_cost_weight_desc': 'total cost = elec_cost_weight * elec_cost + carbon_cost_weight * carbon_cost', 'elec_cost_weight': 0.6, 'elec_cost_weight_desc': 'total cost = elec_cost_weight * elec_cost + carbon_cost_weight * carbon_cost', 'max_power': 100, 'max_power_desc': 'max total charge power in KW'}
Tesla Charging Example (to writer):


Answer JSON:


----------------------------------------------

In [16]:
reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

In [18]:
user_car1.initiate_chat(agentAll, message="Update end time to 7 PM", clear_history=True)

HVC0538 (to Tesla Charging Example):

Update end time to 7 PM

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"HVC0538":{"end_time": 19}
```

--------------------------------------------------------------------------------
{'HVC0538': {'end_time': 19}}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0xbf4f3ad1
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -121.8000000
Presolve removed 56 ro

In [19]:
reset_params_file(params_filepath, params_filepath_backup)
clear_param_backups(params_filepath)

In [20]:
user_car1.initiate_chat(agentIndividual, message="Update end time to 7 PM", clear_history=True)

HVC0538 (to Tesla Charging Example):

Update end time to 7 PM

--------------------------------------------------------------------------------
{'max_charge': 50, 'max_power': 11.5, 'required_charge': 2, 'end_time': 20, 'required_charge_desc': 'Required charge requested by car in KWH e_ri', 'max_charge_desc': 'Max charge that can be supplied to the car in KWH e_mi', 'max_power_desc': 'Max power supported by charger in W', 'end_time_desc': 'Hour by which car should be charged in 24 hour format'}
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
{"end_time": 19}
```

--------------------------------------------------------------------------------
{'end_time': 19}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 column

In [14]:
user_car2.initiate_chat(agent, message="Update required charge to 20", clear_history=True)

ZNV0672 (to Tesla Charging Example):

Update required charge to 20

--------------------------------------------------------------------------------
Tesla Charging Example (to writer):


Answer JSON:


--------------------------------------------------------------------------------
writer (to Tesla Charging Example):

```JSON
"ZNV0672":{"required_charge": 20}
```

--------------------------------------------------------------------------------
{'ZNV0672': {'required_charge': 20}}
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 56 rows, 192 columns and 576 nonzeros
Model fingerprint: 0xf6477c0f
Variable types: 0 continuous, 192 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-01, 4e+01]
  Bounds range     [1e+01, 1e+01]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective -50.5200000
Pres